In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')
import pandas as pd
import numpy as np

import seaborn as sns
sns.set()

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score

from sklearn.metrics import accuracy_score

from sklearn.svm import SVC
from sklearn.model_selection import learning_curve,validation_curve

In [2]:
# load the data
df = pd.read_csv('./Data/Final_18_features_extracted_V2.csv')

In [3]:
X = df.drop(columns=["Region","File Name"],axis=1)
y = df['Region']

In [4]:
X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.3,random_state=17,shuffle=True
)

In [5]:
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=17)

In [6]:
svc = SVC(random_state=17)

In [7]:
results = cross_val_score(svc,X_train,y_train,cv=skf)
results.mean()*100

42.75346592985361

In [8]:
svc.fit(X_train,y_train)

SVC(random_state=17)

In [9]:
# accuracy on the train set:
rfc_pred = svc.predict(X_train)
accuracy_score(y_train,rfc_pred)

0.4362087998738369

In [10]:
# accuracy on the test set
rfc_pred = svc.predict(X_test)
accuracy_score(y_test,rfc_pred)

0.43193524650478293

- Validation Curves

In [11]:
def plot_with_err(x, data, **kwargs):
    mu, std = data.mean(1), data.std(1)
    lines = plt.plot(x, mu, "-", **kwargs)
    plt.fill_between(
        x,
        mu - std,
        mu + std,
        edgecolor="none",
        facecolor=lines[0].get_color(),
        alpha=0.2,
    )
def plot_validation_curve(param_grid,param,estimator):
    val_train, val_test = validation_curve(
    estimator=estimator,
    X=X_train,
    y=y_train,
    param_name=param,
    param_range=param_grid,
    cv=skf,
    scoring="accuracy",
    n_jobs=-1
    )

    plot_with_err(param_grid, val_train, label="training scores")
    plot_with_err(param_grid, val_test, label="validation scores")
    plt.xlabel(param)
    plt.ylabel("accuracy")
    plt.legend()
    plt.grid(True);

- C

In [12]:
# plot_validation_curve(np.linspace(50,200,20),'C',svc)

- Kernel

In [13]:
# plot_validation_curve(['linear','poly','rbf','sigmoid'],'kernel',svc)

- seems like linear is the best kernel for this dataset

In [14]:
svc = SVC(random_state=17,C=100,kernel='linear')
results = cross_val_score(svc,X_train,y_train,cv=skf,n_jobs=-1)
results.mean()*100